In [6]:
import openai, json

client = openai.OpenAI()

system_prompt = """당신은 사용자의 취향을 기억하는 영화 추천 챗봇입니다.

규칙:
1. 사용자가 좋아하는 장르를 대화 내용에서 파악하고 기억하세요.
2. 사용자가 이미 시청한 영화를 기억하세요.
3. 영화를 추천할 때, 이미 본 영화는 절대 추천하지 마세요.
4. 대화 기록을 기반으로 개인화된 추천을 제공하세요.
5. 사용자가 질문한 언어로 답변하세요.
"""

messages = [{"role": "system", "content": system_prompt}]

In [7]:
import requests

BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"


def get_popular_movies():
    response = requests.get(f"{BASE_URL}/movies")
    return response.json()


def get_movie_details(id):
    response = requests.get(f"{BASE_URL}/movies/{id}")
    return response.json()


def get_movie_credits(id):
    response = requests.get(f"{BASE_URL}/movies/{id}/credits")
    return response.json()

In [8]:
FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_movie_credits": get_movie_credits,
}

TOOLS = [
  {
    "type": "function",
    "function": {
      "name": "get_popular_movies",
      "description": "Get the popular movies",
      "parameters": {
        "type": "object",
        "properties": {
        },
        "required": [],
      },
    },
  },
  {
    "type": "function",
    "function": {
      "name": "get_movie_details",
      "description": "Get the details of a movie",
      "parameters": {
        "type": "object",
        "properties": {
          "id": {
            "type": "integer",
            "description": "The id of the movie to get the details of.",
          },
        },
        "required": ["id"],
      },
    },
  },
  {
    "type": "function",
    "function": {
      "name": "get_movie_credits",
      "description": "Get the credits of a movie",
      "parameters": {
        "type": "object",
        "properties": {
          "id": {
            "type": "integer",
            "description": "The id of the movie to get the credits of.",
          },
        },
        "required": ["id"],
      },
    },
  }
]


In [9]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):

    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": json.dumps(result),
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [10]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: 지금 인기 있는 한국영화 알려줘
Calling function: get_popular_movies with {}
Ran get_popular_movies with args {} for a result of [{'adult': False, 'backdrop_path': 'https://image.tmdb.org/t/p/w1280/7HKpc11uQfxnw0Y8tRUYn1fsKqE.jpg', 'genre_ids': [878, 28, 53], 'id': 1236153, 'original_language': 'en', 'original_title': 'Mercy', 'overview': 'In the near future, a detective stands on trial accused of murdering his wife. He has ninety minutes to prove his innocence to the advanced AI Judge he once championed, before it determines his fate.', 'popularity': 379.2699, 'poster_path': 'https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg', 'release_date': '2026-01-20', 'title': 'Mercy', 'video': False, 'vote_average': 7.12, 'vote_count': 535}, {'adult': False, 'backdrop_path': 'https://image.tmdb.org/t/p/w1280/6YjnTRBz704LF1uJ3ZC4wsS9T8r.jpg', 'genre_ids': [28, 80, 53], 'id': 1290821, 'original_language': 'en', 'original_title': 'Shelter', 'overview': 'A man living in self-imposed exile on a